## wine 예제, 하이퍼파라미터 튜닝 후 MLP 딥러닝 분류 
### + 원-핫 인코딩
원-핫 인코딩은 분류에서만 사용됨

In [293]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import tensorflow as tf
import kerastuner as kt
from keras.models import Sequential
from keras.layers import Dense,Input
from tensorflow.keras.utils import to_categorical

### CSV파일 로딩하기

In [295]:
file_path = './wine.csv'
df = pd.read_csv(file_path)

df

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [296]:
print("shape:", df.shape)
print("columns:", df.columns)
# 결측치 확인
print("=== 결측치 현황 ===")
print(df.isnull().sum())

shape: (178, 14)
columns: Index(['Wine', 'Alcohol', 'Malic.acid', 'Ash', 'Acl', 'Mg', 'Phenols',
       'Flavanoids', 'Nonflavanoid.phenols', 'Proanth', 'Color.int', 'Hue',
       'OD', 'Proline'],
      dtype='object')
=== 결측치 현황 ===
Wine                    0
Alcohol                 0
Malic.acid              0
Ash                     0
Acl                     0
Mg                      0
Phenols                 0
Flavanoids              0
Nonflavanoid.phenols    0
Proanth                 0
Color.int               0
Hue                     0
OD                      0
Proline                 0
dtype: int64


In [297]:
# 레이블 분포 확인
print("\n=== 레이블 분포 ===")
print(df['Wine'].value_counts())


=== 레이블 분포 ===
Wine
2    71
1    59
3    48
Name: count, dtype: int64


### 특성과 타켓(레이블) 분리
X: 특성
y: 타겟

In [299]:
X = df.drop('Wine', axis=1).values # Pandas DataFrame을 NumPy 배열로 변환, wine 제외
y = df['Wine'].values

# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 타겟변수 (Y) 원핫 인코딩 수행 
타겟변수 (Y)를 다중 클래스 분류에 적합한 형태로 변환

In [301]:
# 레이블 원-핫 인코딩 (분할 후 수행)
y_train_onehot = to_categorical(y_train - 1)  # 훈련 데이터 인코딩
y_test_onehot = to_categorical(y_test - 1)    # 테스트 데이터 변환

#### 데이터 스케일링

In [303]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 훈련 데이터로 학습
X_test_scaled = scaler.transform(X_test)        # 테스트 데이터 변환

In [304]:
# 데이터셋 Shape 확인
print("\n=== 데이터셋 Shape ===")
print("X_train shape:", X_train_scaled.shape)
print("X_test shape:", X_test_scaled.shape)
print("y_train shape:", y_train_onehot.shape)
print("y_test shape:", y_test_onehot.shape)


=== 데이터셋 Shape ===
X_train shape: (142, 13)
X_test shape: (36, 13)
y_train shape: (142, 3)
y_test shape: (36, 3)


### 하이퍼파라미터 튜닝 

#### 모델 구조 정의
hp.int() : 정수 범위 내에서 최적의 노드 수 탐색

hp.choice(): 옵티마이저(adam/sgd) 선택

In [307]:
# 원-핫 인코딩 후 클래스 수 계산
num_classes = y_train_onehot.shape[1]  # 클래스 개수 (예: 3)

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],))) # 입력층(Input): 특성 수만큼 노드 설정
    # 첫번째 은닉층: 32~128개의 노드 중 최적값 탐색 (32단위)
    model.add(Dense(hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu'))
    # 두번째 은닉층: 16~64개의 노드 중 최적값 탐색 (16단위)
    model.add(Dense(hp.Int('units_2', min_value=16, max_value=64, step=16), activation='relu'))
    # 출력층: 클래스 수만큼 노드 + 소프트맥스 활성화(다중 분류)
    model.add(Dense(num_classes, activation='softmax'))


    # 컴파일: 옵티마이저(adam, sgd)탐색
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
        loss='categorical_crossentropy', # 다중 클래스 분류용 손실 함수
        metrics=['accuracy'] # 성능 확인 지표 (정확도 + 정밀도)
    )
    
    return model

#### 튜닝 디렉토리 삭제

In [309]:
import shutil
shutil.rmtree("my_tuning_dir", ignore_errors=True)

#### Keras Tuner 설정 (하이퍼파라미터 탐색 설정)

In [311]:
# 무작위로 하이퍼파라미터 조합을 시도함
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy', # 검증 정확도 최대화
    max_trials=10, # 총 10번 실험
    directory='my_tuning_dir', # 결과 저장 경로
    project_name='wine_classification', # 이름
    overwrite=True # 기존 데이터 덮어쓰기 (KeyError해결)
)

#### 하이퍼파라미터 탐색 수행
- epochs=50 : 각 실험별 50회 학습
- batch_size=32 : 미니 배치 크기 32
- validation_split=0.2 : 훈련 데이터의 20%를 검증용으로 사용

In [313]:
tuner.search(X_train_scaled, y_train_onehot, epochs=50, batch_size=32, validation_split=0.2)

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.9655172228813171

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 36s


#### 최적의 하이퍼파라미터 출력

In [315]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest Hyperparameters:")
print(f"첫 번째 은닉층 노드 수: {best_hps.get('units_1')}")
print(f"두 번째 은닉층 노드 수: {best_hps.get('units_2')}")
print(f"Optimizer: {best_hps.get('optimizer')}")


Best Hyperparameters:
첫 번째 은닉층 노드 수: 32
두 번째 은닉층 노드 수: 64
Optimizer: adam


#### 최적의 하이퍼파라미터로 MLP 딥러닝

In [317]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train_onehot, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5091 - loss: 1.0524 - val_accuracy: 0.5862 - val_loss: 0.8545
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6299 - loss: 0.7984 - val_accuracy: 0.7241 - val_loss: 0.6402
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7519 - loss: 0.6325 - val_accuracy: 0.8276 - val_loss: 0.4997
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9052 - loss: 0.4494 - val_accuracy: 0.8276 - val_loss: 0.4070
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8829 - loss: 0.3750 - val_accuracy: 0.8276 - val_loss: 0.3427
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9196 - loss: 0.3343 - val_accuracy: 0.8276 - val_loss: 0.2924
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9144 - loss: 0.2779 - val_accuracy: 0.8966 - val_loss: 0.2534
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9367 - loss: 0.2597 - val_accuracy: 0.9310 - val_loss: 0.2208


#### 테스트 데이터 평가

In [319]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_onehot)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 1.0000 - loss: 0.0112

Test Loss: 0.0110
Test Accuracy: 1.0000


#### 예측 수행 및 결과 확인

In [321]:
y_pred_probs = model.predict(X_test_scaled)  # 예측 확률 계산
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # 예측된 클래스 인덱스 (가장 확률이 높은 값 선택)
y_true_classes = np.argmax(y_test_onehot, axis=1)  # 실제 클래스 인덱스

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x3704784c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


#### 성능지표(f1_score, precision, recall) 확인

In [323]:
f1 = f1_score(y_true_classes, y_pred_classes, average='macro')
precision = precision_score(y_true_classes, y_pred_classes, average='macro')
recall = recall_score(y_true_classes, y_pred_classes, average='macro')

print("\n=== CNN 성능 지표 ===")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


=== CNN 성능 지표 ===
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000


#### 혼동 행렬 출력

In [325]:
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
